# Simulating the SSH model

Here we simulate the SSH model using the Pulser backend.
We follow the work of De Léséleuc et al. in ["Observation of a symmetry-protected topological phase of interacting bosons with Rydberg atoms"](https://www.science.org/doi/10.1126/science.aav9105) (available also on [arxiv](https://arxiv.org/abs/1810.13286)).

Using a Microwave channel, the interaction Hamiltonian is given by

```{math}
H = \sum_{i=1}^N \sum_{j<i} C_{ij} (\sigma_i^+ \sigma_j^- + \sigma_i^- \sigma_j^+)
```
where 
{math}`C_{ij} = \frac{C_3(1-3\cos^2\theta_{ij})}{R_{ij}^3}`.
Here {math}`R_{ij}` is the distance between qubits {math}`i` and {math}`j`, {math}`R_{ij}=|\textbf{x}_i-\textbf{x}_j|`, and {math}`\theta_{ij}` is the angle between {math}`\textbf{x}_i-\textbf{x}_j` and the magnetic field.
See the [Pulser docs](https://pulser.readthedocs.io/en/stable/tutorials/xy_spin_chain.html) for more information.

By appropriate choice of the qubit geometry we can approximate the [SSH model](https://en.wikipedia.org/wiki/Su%E2%80%93Schrieffer%E2%80%93Heeger_model) for hard bosons.
For example, when 
$$ \theta_{ij} = \cos^{-1}\sqrt{1/3}$$
there will be no interaction between qubits {math}`i` and {math}`j`. We use this to model the SSH model below.


In [ ]:
import qse
import pulser
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
angle = np.arccos(np.sqrt(1.0 / 3.0)) * 180 / np.pi
print(f"Angle is: {angle:.2f} degrees")

We create two chains of qubits, A and B.
By rotating them by the angle above, there will be no interactions between qubits on the same chain.

In [ ]:
lattice_spacing = 12
repeats = 4

qbits1 = qse.lattices.chain(lattice_spacing=lattice_spacing, repeats=repeats)
qbits1.labels = [f"A{i}-{i}" for i in range(repeats)]

qbits2 = qse.lattices.chain(lattice_spacing=lattice_spacing, repeats=repeats)
qbits2.labels = [f"B{i}-{i+repeats}" for i in range(repeats)]
qbits2.translate((lattice_spacing * 0.5, 8, 0))

qbits_ssh = qbits1 + qbits2
qbits_ssh.rotate(
    90 - angle
)  # by rotating to this angle interactions in the A & B chains will cancel.

qbits_ssh.draw(show_labels=True, units="µm")

We can verify that there are no interactions in the same chain by computing the couplings between qubits.

In [ ]:
magnetic_field = np.array([0.0, 1.0, 0.0])
c3 = 3700


def compute_hamilton_coef(i, j):
    """Compute the hamiltonian coefficient between qubits i & j."""
    r = qbits_ssh.positions[i] - qbits_ssh.positions[j]
    d = np.linalg.norm(r)
    r /= d
    cos_theta = np.dot(r, magnetic_field)
    return c3 * (1 - 3 * cos_theta**2) / d**3


couplings = [
    [compute_hamilton_coef(i, j) for j in range(i + 1, qbits_ssh.nqbits)]
    for i in range(qbits_ssh.nqbits - 1)
]
couplings = [j for i in couplings for j in i]
coupling_mat = np.zeros((qbits_ssh.nqbits, qbits_ssh.nqbits))
coupling_mat[np.triu_indices(qbits_ssh.nqbits, 1)] = couplings
couplings
for i in range(1, qbits_ssh.nqbits):
    print(f"Coupling between qbits 0 & {i}:", coupling_mat[0, i])

In [ ]:
# Let's visualize the couplings
bonds = np.abs(coupling_mat)
bonds /= np.max(bonds)
qbits_ssh.draw(show_labels=True, units="µm")
for i in range(0, qbits_ssh.nqbits - 1):
    for j in range(i + 1, qbits_ssh.nqbits):
        xs_ij = qbits_ssh.positions[[i, j]]
        plt.plot(xs_ij[:, 0], xs_ij[:, 1], c="k", alpha=bonds[i, j], zorder=-1)

Thus we see that there are no couplings between qubits in the same chain, and hence we have a SSH model.
Finally we can visualize the Hamiltonian.

In [ ]:
simple_pulse = pulser.Pulse.ConstantPulse(10, 0.0, 0.0, 0.0)
pcalc_ssh = qse.calc.Pulser(
    qbits=qbits_ssh,
    amplitude=simple_pulse.amplitude,
    detuning=simple_pulse.detuning,
    channel="mw_global",
    magnetic_field=magnetic_field,
)
pcalc_ssh.build_sequence()
pcalc_ssh.calculate()

In [ ]:
ham = pcalc_ssh.sim.get_hamiltonian(0).full()
fig = qse.visualise.view_matrix(ham.real, vcenter=0.0)

## Version

In [ ]:
qse.utils.print_environment()